In [1]:
# Yolov5をインストールする

!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

%load_ext tensorboard

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0 (Tesla P100-PCIE-16GB)


In [2]:
data_path = "/kaggle/input/snakegameyolodataset"

In [3]:
import os
os.path.exists("/kaggle/working/yolov5")

True

In [4]:
# 学習画像を学習する
# --imgは画像の幅・高さのうち大きい方のサイズです．大きさを間違えて入力しても特に変化がなかったのであまり気にする必要はないと思います．
# --batchは一度に何枚の画像を学習するかです．--epochsは--batchで指定した枚数を学習することを一回とした時に何回その学習を行うかです．以下の--batch 10 --epochs 100という場合だと一度に10枚学習してそれを100回繰り返すという意味なので，学習画像が全部で100枚しかないとすると1枚の画像につき10回は繰り返し学習をすることになります．
# --batchを大きくしすぎると一度の計算量が大きくなってColabがリミットを計算量のリミットを超過してしばらく使えなくなってしまいます．小さくしすぎると計算時間がかかるし，学習が非効率的になってしまいます．
# --epochを小さくしすぎると最適値に到達するまでに必要な学習の回数に足りなくなってしまいます．大きくしすぎると最適値に到達した後にも永遠と学習し続けてしまうので余計な時間がかかってしまいます．
# 私がやったときは学習画像が100枚ぐらいだったので--batch 10 --epochs 100ぐらいがちょうど良かったです．
# --dataは学習の際に必要な情報（学習画像があるディレクトリのパスだったりとかの情報）が書かれているYamlファイルのパスを指定します．train.yamlのパスを書いてください．
# --weightsはyolov5s.pt等を指定しておけばOKだと思います．
# --cacheは実行の際の読み取ったものをキャッシュしてくれるとかなんとからしいですが，特に書いても書かなくても問題ないと思います．書いておいたら再度学習するときにより早く学習できる感じです．

# うまく実行できた場合は学習の最後に以下の文と同じようなものが出力されると思います．
##################################################################################################################################
# 100 epochs completed in 0.273 hours.
# Optimizer stripped from runs/train/exp/weights/last.pt, 14.4MB  ←last.ptとは，一番最後に学習をしたときに得られる重みが書かれているファイルです．
# Optimizer stripped from runs/train/exp/weights/best.pt, 14.4MB  ←best.ptとは，学習をして得られた重みの中で一番最適な重みが書かれているファイルです．最適な重みが書かれたものなのでlast.ptではなく，当然こちらの重みファイルの方を以下では使っていくことになります．ここでは「runs/train/exp/weights/best.pt」がbest.ptのパスです．

# Validating runs/train/exp/weights/best.pt...
# Fusing layers... 
# Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
#                Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 5/5 [00:04<00:00,  1.09it/s]
#                  all        100        143      0.812      0.818      0.891      0.485
# Results saved to runs/train/exp
##################################################################################################################################

!python train.py --img 1320 --batch 4 --epochs 1000 --data /kaggle/input/snakegameyolodataset/data.yaml --weights yolov5x.pt --cfg yolov5x.yaml --cache 

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
train: weights=yolov5x.pt, cfg=yolov5x.yaml, data=/kaggle/input/snakegameyolodataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=1000, batch_size=4, imgsz=1320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-203-g6371de8 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparam

In [5]:
%tensorboard --logdir runs/train

In [6]:
# 精度評価
# このセルでは最適値の重み（best.pt）を使って，テスト画像を検出したときの精度（mAP）を出します．客観的に精度を数値化できるので精度比較の際の指標として使います．
# mAPについては　https://pystyle.info/how-to-calculate-object-detection-metrics-map/　または　https://qiita.com/cv_carnavi/items/08e11426e2fac8433fed　の説明がわかりやすいです．ただ，Yolov5が自動でmAPの数値を出してくれるので左記の記事にならって自分で計算する必要はありません．
# --weightsにはbest.ptのパスを入れてください．
# --imgは学習のときと同じく画像のサイズです．
# # --dataは精度評価の際に必要な情報（テスト画像があるディレクトリのパスだったりとかの情報）が書かれているYamlファイルのパスを指定します．test.yamlのパスを書いてください．

# 最後に以下と同じような結果が出力されると思います．この出力部分はスクショしておいてください．なぜかこの時にしか見れないので出力結果全体をスクショしておいてGoogleドライブに保存しておいた方がいいです．
# mAP@.5 は閾値が0.5の時のmAPの数値．
# mAP@.5:.95は閾値が0.5, 0.55, 0.6, 0.65 ... 0.85, 0.9, 0.95のそれぞれの時のmAPの数値の平均値．
##################################################################################################################################
# val: data=/content/drive/MyDrive/test.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=600, conf_thres=0.001, iou_thres=0.6, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False
# YOLOv5 🚀 v6.0-28-g79d8f1f torch 1.9.0+cu111 CUDA:0 (Tesla K80, 11441.1875MB)

# Fusing layers... 
# Model Summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
# WARNING: --img-size 600 must be multiple of max stride 32, updating to 608
# val: Scanning '/content/drive/MyDrive/dataset/test/labels.cache' images and labels... 1055 found, 0 missing, 0 empty, 0 corrupted: 100% 1055/1055 [00:00<?, ?it/s]
#                Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 33/33 [02:22<00:00,  4.31s/it]
#                  all       1055       1652      0.427      0.326      0.287     0.0991  ←このmAP@.5とmAP@.5:.95の数値がmAPの数値です．
# Speed: 0.4ms pre-process, 15.6ms inference, 2.0ms NMS per image at shape (32, 3, 608, 608)
# Results saved to runs/val/exp3  ←結果がこのディレクトリに保存されたと言っています．ここでは結果が保存されているディレクトリのパスは「runs/val/exp3」です．
##################################################################################################################################

!python val.py --weights /kaggle/working/yolov5/runs/train/exp/weights/best.pt --img 600 --data /kaggle/input/snakegameyolodataset/data.yaml

val: data=/kaggle/input/snakegameyolodataset/data.yaml, weights=['/kaggle/working/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=600, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.2-203-g6371de8 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86186872 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ --img-size 600 must be multiple of max stride 32, updating to 608
val: Scanning '/kaggle/input/snakegameyolodataset/data/val/labels' images and la
val: WARNING ⚠️ Cache directory /kaggle/input/snakegameyolodataset/data/val is not writeable: [Errno 30] Read-only file system: '/kaggle/input/snakegameyolodataset/data/val/labels.cache.npy'
                 Class     Images  Instances          

In [7]:
!python detect.py --source /kaggle/input/snakegameyolodataset/data/images --weights /kaggle/working/yolov5/runs/train/exp/weights/best.pt --img 1320 --conf 0.1

detect: weights=['/kaggle/working/yolov5/runs/train/exp/weights/best.pt'], source=/kaggle/input/snakegameyolodataset/data/images, data=data/coco128.yaml, imgsz=[1320, 1320], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v6.2-203-g6371de8 Python-3.7.12 torch-1.11.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
YOLOv5x summary: 322 layers, 86186872 parameters, 0 gradients, 203.8 GFLOPs
WARNING ⚠️ --img-size [1320, 1320] must be multiple of max stride 32, updating to [1344, 1344]
image 1/99 /kaggle/input/snakegameyolodataset/data/images/image_257.png: 1344x1344 1 snake-head, 1 apple, 2 snake-bodys, 123.3ms
image 2/99 /kaggle/input/snakegameyolodataset/data/images/ima